In [1]:
!head -1 stackoverflow.10kk.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


In [2]:
%%time
!wc -l stackoverflow.10kk.tsv

10000000 stackoverflow.10kk.tsv
CPU times: user 1.08 s, sys: 280 ms, total: 1.36 s
Wall time: 34.8 s


In [3]:
import os
from tqdm import tqdm
from time import time
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

PATH_TO_DATA = ('D:/Anaconda/PyCharm1/')

In [4]:
!python preprocess_sof.py stackoverflow.10kk.tsv stackoverflow.vw

10000000it [02:19, 71802.03it/s]
4389054 lines selected, 15 lines corrupted.


In [5]:
%%time
!wc -l stackoverflow.vw

4389054 stackoverflow.vw
CPU times: user 550 ms, sys: 170 ms, total: 720 ms
Wall time: 18.2 s


In [6]:
!gzip stackoverflow.10kk.tsv

In [ ]:
# делим файл на 3 части по 1.463.018 строк
# -a позволяет контролировать длину суффикса(в данном случае длина первого суффикса = 1)
# -d позволяет использовать числовой суффикс
!split -l 1463018 -a 1 - d stackoverflow.vw
# на выходе получим: x0, x1, x2. P.S. переименуем их ручками.


# векторы ответов
!cut -c 1 stackoverflow_valid.vw > stackoverflow_valid_labels.txt
!cut -c 1 stackoverflow_test.vw > stackoverflow_test_labels.txt

#ЭТО НЕ НУЖНО, ОТВЕТЫ НЕЛЬЗЯ УДАЛЯТЬ, но код просто так на будущее
# удаляем ответы из самих файлов(первые 4 строки), оставляя только текст
!sed -r 's/.{4}//' stackoverflow_train.vw

In [ ]:
./vw --oaa 10 # 10-классовая задача
-d 'D:/Anaconda/PyCharm1/stackoverflow_train.vw' # путь к данным
-f -'D:/Anaconda/PyCharm1/vw_model_Xngram_Xpasses.vw'# путь к модели
-b 28 # бит для хэширования
--random_seed 18 # сид
--ngram X # Х = 1, 2, 3
--passes X # X = 1, 3, 5
-c # использование кэша(для passes= 3, 5)

In [ ]:
./vw -t -i 'D:/Anaconda/PyCharm1/vw_model_Xngram_Xpasses.vw' # обученная модель
-d 'D:/Anaconda/PyCharm1/stackoverflow_valid.vw' # какую модель хотим проверить
-p 'D:/Anaconda/PyCharm1/vw_model_1ngram_1passes_pred.csv' # куда хотим записать предсказания
--random_seed 18

In [29]:
%%time
# Доля правильных ответов на отложенной выборке:
accuracy_list = []
for i in range(1, 4):
    for j in range(1, 7, 2):
        vw_pred = np.loadtxt(os.path.join(PATH_TO_DATA, 'vw_model_{}ngram_{}passes_pred.csv'.format(i, j)))
        valid_labels = np.loadtxt(os.path.join(PATH_TO_DATA, 'stackoverflow_valid_labels.txt'))
        score = accuracy_score(valid_labels, vw_pred)
        accuracy_list.append(score)
        print('Ngram =', i, 'Passes =', j, 'Accuracy =', score)
print('Best accuracy =', max(accuracy_list))

Ngram = 1 Passes = 1 Accuracy = 0.894877574985
Ngram = 1 Passes = 3 Accuracy = 0.893934319332
Ngram = 1 Passes = 5 Accuracy = 0.893015670347
Ngram = 2 Passes = 1 Accuracy = 0.910350385299
Ngram = 2 Passes = 3 Accuracy = 0.906772165483
Ngram = 2 Passes = 5 Accuracy = 0.908225326004
Ngram = 3 Passes = 1 Accuracy = 0.907940298752
Ngram = 3 Passes = 3 Accuracy = 0.90553773091
Ngram = 3 Passes = 5 Accuracy = 0.90534839626
Best accuracy = 0.910350385299
Wall time: 3min 10s


1:
Биграммы и 1 проход по выборке


In [ ]:
./vw -t -i 'D:/Anaconda/PyCharm1/vw_model_2ngram_1passes.vw' # обученная модель
-d 'D:/Anaconda/PyCharm1/stackoverflow_test.vw' # какую модель хотим проверить
-p 'D:/Anaconda/PyCharm1/vw_model_2ngram_1passes_pred_for_test.csv' # куда хотим записать предсказания
--random_seed 18

In [17]:
vw_test = np.loadtxt(os.path.join(PATH_TO_DATA, 'vw_model_2ngram_1passes_pred_for_test.csv'))
test_labels = np.loadtxt(os.path.join(PATH_TO_DATA, 'stackoverflow_test_labels.txt'))
best_accuracy_1 = accuracy_score(test_labels, vw_test)
print(best_accuracy_1)

0.910435141605


In [30]:
print('Difference =', round((best_accuracy_1 - max(accuracy_list)) * 100, 3))

Difference = 0.008


2: Результаты почти одинаковы – отличаются меньше чем на 0.5%

In [ ]:
# соеденим два файла
!cd 'D:/Anaconda/PyCharm1/'
!cat stackoverflow_train.vw stackoverflow_valid.vw > stackoverflow_t_v.vw

In [ ]:
# обучим модель на данных вдвое больше
./vw --oaa 10 -d 'D:/Anaconda/PyCharm1/stackoverflow_t_v.vw'
-f -'D:/Anaconda/PyCharm1/vw_best_model.vw'
-b 28 --random_seed 18 --ngram 2 --passes 1

In [ ]:
# сделаем прогноз
./vw -t -i 'D:/Anaconda/PyCharm1/vw_best_model.vw' # обученная модель
-d 'D:/Anaconda/PyCharm1/stackoverflow_test.vw' # какую модель хотим проверить
-p 'D:/Anaconda/PyCharm1/vw_best_model_pred_for_test.csv' # куда хотим записать предсказания
--random_seed 18

In [28]:
vw_test = np.loadtxt(os.path.join(PATH_TO_DATA, 'vw_best_model_pred_for_test.csv'))
test_labels = np.loadtxt(os.path.join(PATH_TO_DATA, 'stackoverflow_test_labels.txt'))
best_accuracy_2 = accuracy_score(test_labels, vw_test)
print(best_accuracy_2)
print('Difference =', round((best_accuracy_2 - best_accuracy_1) * 100, 3))

0.913806255289
Difference = 0.337


3: 0.4% 
